<a href="https://colab.research.google.com/github/aekanun2020/2022-PSU-Data-Science-for-Tourism-Course2/blob/main/Starter-2022_Prediction_of_Customer_Behavior_onColab-ClassRev1-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up Spark Environment

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
from pyspark.sql import SparkSession

In [5]:
sc = spark.sparkContext

In [6]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

# Libraries and Modules Installation

In [7]:
#Spark
from pyspark.sql import functions as sparkf
from pyspark.sql import Window as W
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

#Python
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
from functools import reduce

# 1. Business Understanding

**Problem Statement-A: ธุรกิจต้องการได้รับ Machine Learning Model ที่ทำให้ทราบล่วงหน้าว่า ลูกค้าที่กำลังเลือกดูสินค้าอยู่ จะซื้อหรือไม่ซื้อสินค้าหรือไม่ (Binary Classification)**

**Problem Statement-B: ธุรกิจต้องการได้รับ Machine Learning Model ที่ทำให้ทราบล่วงหน้าว่า ลูกค้าที่กำลังเลือกดูสินค้าอยู่ จะมีพฤติกรรมสุดท้าย (Last hitAction_type) เป็นอะไร เช่น ไม่ซื้อโดยมีพฤติกรรมสุดท้ายเป็นหยิบของใส่ตะกร้าไว้เฉยๆ หรือ ไม่ซื้อโดยมีพฤติกรรมอื่นๆ หรือซื้อสินค้า (Complete purchase)  (Multiclasses Classification)**

**Project Objectitves: (1) ธุรกิจต้องการ Offer อะไรสักอย่างให้ลูกค้าตัดสินใจซื้อได้รวดเร็วขึ้น - มุ่งคนที่ตั้งใจซื้ออยู่แล้ว (2) ธุรกิจต้องการให้เกิดการซื้อเพิ่ม (Up-Selling, Cross-Selling) - มุ่งคนที่ตั้งใจซื้ออยู่แล้ว ให้ซื้อมากขึ้น**

*** Analytics Approach: A, B: Classification => Possible types of Algorithm: Supervised, Information-based, Error-based ***

**Requirement from Business Perspective: (1) ยังไม่ชัดเจนเรื่องข้อมูล รวมถึงคุณภาพข้อมูล (2) ปัจจัยที่มีอิทธิพลต่อการตัดสินใจซื้อหรือเลือกใช้บริการมีอะไรบ้าง และอะไรที่ไม่เกี่ยวข้องกับโจทย์นี้โดยตรงบ้าง**

In [8]:
business_NotRequired_col_list = [\
                        #'sourceKeyword'\
                        #,'fullVisitorId'\
                        #,'isVideoAd'\
                        #,'date'\
                        #,'isTrueDirect'\
                        #,'hitHour'\
                        #,'hitMin'\
                        #,'isClick'\
                        #,'productListName'\
                        #,'isImpression'\
                        #,'sessionQualityDim'\
                        #'timeOnScreen'\
                        #,'hitReferer'\
                        #,'medium'\
                        #,'hitType'
                        ]

business_Required_col_list = [\
                        #'sourceKeyword'\
                        #,'fullVisitorId'\
                        #,'isVideoAd'\
                        #,'date'\
                        #,'isTrueDirect'\
                        #,'hitHour'\
                        #,'hitMin'\
                        #,'isClick'\
                        #,'productListName'\
                        #,'isImpression'\
                        #,'sessionQualityDim'\
                        #'timeOnScreen'\
                        #,'hitReferer'\
                        #,'medium'\
                        #,'hitType'
                        ]

**Success Criteria: ต้องการให้มีการทดลองหลายๆ รอบ แล้วมีการทดสอบกับข้อมูลจริงโดยทีมนักการตลาด ซึ่งหากโมเดลไหน นักการตลาดพอใจก็จะเลือกโมเดลนั้น ในเบื้องต้นกำหนดไว้ว่า โมเดลจะต้องมีค่า Recall ไม่น้อยกว่า 0.7 จึงจะนำเข้าสู่กระบวนการทดสอบโดยนักการตลาดได้**

In [9]:
recallThreshold = 0.7

# 2. Data Understanding

### **2.1 Data Collection and Previewing Data**

In [10]:
! wget https://storage.googleapis.com/class25jan2022/share/testfunnel.csv

--2022-03-04 12:41:37--  https://storage.googleapis.com/class25jan2022/share/testfunnel.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.210.128, 173.194.211.128, 173.194.213.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.210.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1501012 (1.4M) [text/csv]
Saving to: ‘testfunnel.csv’

testfunnel.csv      100%[===================>]   1.43M  --.-KB/s    in 0.01s   

2022-03-04 12:41:37 (121 MB/s) - ‘testfunnel.csv’ saved [1501012/1501012]



In [11]:
! head -3 testfunnel.csv

source,isTrueDirect,sourceKeyword,medium,isVideoAd,fullVisitorId,visitId,date,newVisits,hitReferer,hitType,hitAction_type,hitNumber,hitHour,hitMin,timeMicroSec,v2ProductName,productListName,isClick,isImpression,sessionQualityDim,timeOnScreen,timeOnSite,totalTransactionRevenue
google,true,(not provided),organic,,4988612949713423910,1489617360,20170315,,,PAGE,0,28,15,46,608771,Google Accent Insulated Stainless Steel Bottle,Category,,true,,,1897,
google,true,(not provided),organic,,4988612949713423910,1489617360,20170315,,,PAGE,0,27,15,46,605064,Google Pocket Bluetooth Speaker,Category,,true,,,1897,


In [12]:
! wc -l testfunnel.csv

10000 testfunnel.csv


In [15]:
## Spark ทำการ Load & Convert .csv เป็น Spark's DataFrame
source_df = spark.read.option('header','true')\
.option("inferSchema" , "true")\
.format('csv').load('./testfunnel.csv')

### 2.2 EDA